# Prerequisites

In [37]:
import pandas as pd
import numpy as np
from collections import defaultdict
import torch

# Dataset

In [45]:
file_path = r"C:\Users\Asus\Downloads\TWOSIDES\TWOSIDES.csv"

In [ ]:
df = pd.read_csv(file_path, dtype=str)

In [56]:
df.head()

,drug_1_rxnorn_id,drug_1_concept_name,drug_2_rxnorm_id,drug_2_concept_name,condition_meddra_id,condition_concept_name,A,B,C,D,PRR,PRR_error,mean_reporting_frequency
0,10355,Temazepam,136411,sildenafil,10003239,Arthralgia,7,149,24,1536,2.91667,0.421275,0.0448718
1,1808,Bumetanide,7824,Oxytocin,10003239,Arthralgia,1,13,2,138,5.0,1.19224,0.0714286
2,221147,POLYETHYLENE GLYCOL 3350,5521,Hydroxychloroquine,10003239,Arthralgia,6,103,20,1070,3.0,0.454505,0.0550459
3,10324,Tamoxifen,8640,Prednisone,10012735,Diarrhoea,18,123,35,1375,5.14286,0.276271,0.12766
4,10355,Temazepam,136411,sildenafil,10012735,Diarrhoea,2,154,37,1523,0.540541,0.721093,0.0128205


In [58]:
df.shape

(42920391, 13)

In [57]:
df.isnull().sum()

drug_1_rxnorn_id            0
drug_1_concept_name         0
drug_2_rxnorm_id            0
drug_2_concept_name         0
condition_meddra_id         0
condition_concept_name      0
A                           0
B                           0
C                           0
D                           0
PRR                         0
PRR_error                   0
mean_reporting_frequency    0
dtype: int64

In [60]:
df.duplicated().sum()

np.int64(1000)

In [64]:
df = df.drop_duplicates(subset=['drug_1_rxnorn_id', 'drug_2_rxnorm_id', 'condition_meddra_id'])

In [65]:
df.duplicated().sum()

np.int64(0)

In [66]:
df.shape

(42919391, 13)

In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 42919391 entries, 0 to 42920390
Data columns (total 13 columns):
 #   Column                    Dtype 
---  ------                    ----- 
 0   drug_1_rxnorn_id          object
 1   drug_1_concept_name       object
 2   drug_2_rxnorm_id          object
 3   drug_2_concept_name       object
 4   condition_meddra_id       object
 5   condition_concept_name    object
 6   A                         object
 7   B                         object
 8   C                         object
 9   D                         object
 10  PRR                       object
 11  PRR_error                 object
 12  mean_reporting_frequency  object
dtypes: object(13)
memory usage: 4.5+ GB


In [ ]:
df = df.copy()  # Make a deep copy to break the link to the original DataFrame

for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

C:\Users\Asus\AppData\Local\Temp\ipykernel_19948\3926647545.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = pd.to_numeric(df[col], errors='coerce')


In [ ]:
# Filter low-signal pairs and removing rare reports
df = df[(df['PRR'] > 1.5) & (df['mean_reporting_frequency'] > 0.01)].copy()